In [22]:
from haystack.nodes import PDFToTextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import PineconeDocumentStore
from dotenv import dotenv_values
from pathlib import Path
from haystack.pipelines import Pipeline
import pandas as pd
import numpy as np
import openai
import pinecone
import os

In [23]:
config = dotenv_values('api_keys.env')
openai.api_key = config['OPENAI_API_KEY']
pinecone_api_key = config['PINECONE_API_KEY']
pinecone_env = config['PINECONE_ENV']

In [24]:
pinecone.init(api_key=pinecone_api_key,
              environment=pinecone_env)

# pinecone.create_index(name='diw-test-index',
#                        dimension=1536)
pinecone.list_indexes()

c:\Users\Dom\VSCodeProjects\ResearchChatBot\.venv\lib\site-packages\pinecone\core\client\rest.py:45: DeprecationWarning: HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).
  return self.urllib3_response.getheader(name, default)
C:\Users\Dom\AppData\Local\Temp\ipykernel_18292\2381031466.py:6: ResourceWarning: unclosed <ssl.SSLSocket fd=5552, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.13', 58756), raddr=('34.140.58.85', 443)>
  pinecone.list_indexes()


['testing']

In [25]:
document_store = PineconeDocumentStore(
    api_key=pinecone_api_key,
    environment=pinecone_env,
    similarity="cosine",
    index='testing',
    embedding_dim=1536
)

c:\Users\Dom\VSCodeProjects\ResearchChatBot\.venv\lib\site-packages\haystack\document_stores\pinecone.py:224: ResourceWarning: unclosed <ssl.SSLSocket fd=5552, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.13', 58758), raddr=('34.140.58.85', 443)>
  if index not in pinecone.list_indexes():


In [42]:
df = pd.read_pickle('diw.pkl')
print(df.head())

                                                                             title   
0  Tuning the 3d printability and thermomechanical properties of radiation shields  \
1  Rheological behavior of titania ink and mechanical properties of titania cer...   
2  Unconventional Application of Direct Ink Writing: Surface Force-Driven Patte...   
3  Rheological Study and Printability Investigation of Titania Inks for Direct ...   
4                       Robocasting of structural ceramic parts with hydrogel inks   

                author  year   
0  Brounstein, Zachary  2021  \
1            Chen, Tao  2019   
2     Cordonier, G. J.  2020   
3    Dolganov, Aleksei         
4        Feilden, Ezra  2016   

                                                                          abstract   
0  Additive manufacturing, with its rapid advances in materials science, allows...  \
1  In this paper, high solid loading titania ceramic ink containing sodium hexa...   
2  A new concept for the direct 

In [27]:
meta_data = df.to_dict('records')

In [28]:

converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["de","en"]
)

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0
)

# document_store = InMemoryDocumentStore(use_bm25=True)




In [29]:
def get_embeddings(text: str, model="text-embedding-ada-002") -> list[float]:
    embedding = openai.Embedding.create(
        input=text,
        model=model
    )["data"][0]["embedding"]

    return embedding

In [33]:
def get_embeddings_for_pinecone(text: str, model="text-embedding-ada-002") -> list[str]:
    embedding = openai.Embedding.create(
        input=text,
        model=model
    )["data"][0]["embedding"]

    return [str(i) for i in embedding]

In [36]:
input_string = 'I am the terminator'

x = get_embeddings_for_pinecone(text=input_string)
print(type(x[0]))

<class 'str'>


In [30]:
def create_prompt(dataframe, df_column, question):

    content = []
    dataframe[df_column].apply(lambda x: content.append(str(x).replace(",", '').replace("'", '')))

    header = """Answer the question as truthfully as possible using the provided context, and if the answer
     is not contained within the text below, say "I don't know."\n\nContext:\n"""

    header_total = str(header + str(content)).replace('[', '').replace(']', '')

    return header_total + "\n\n Q: " + question + "\n A:"

In [37]:
for count, filename in enumerate(os.listdir("pdfs/")):
    if count < 1:
        print(f'Working on {filename}')
        doc = converter.convert(file_path=Path("pdfs/"+str(filename)), meta=meta_data[count])
        processed_docs = processor.process(doc)
        
        for doc_count in range(len(processed_docs)):
            embedding_meta_data = get_embeddings_for_pinecone(text=processed_docs[doc_count].content)
            
            processed_docs[doc_count].meta['embedding'] = embedding_meta_data

        document_store.write_documents(processed_docs)
    else:
        break

Working on Brounstein_2021.pdf


Writing Documents:   0%|          | 0/67 [00:00<?, ?it/s]c:\Users\Dom\VSCodeProjects\ResearchChatBot\.venv\lib\site-packages\pinecone\core\client\rest.py:45: DeprecationWarning: HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).
  return self.urllib3_response.getheader(name, default)
Writing Documents: 96it [00:02, 32.92it/s]                        


In [40]:
index = pinecone.Index('testing')

In [41]:
index.describe_index_stats()

c:\Users\Dom\VSCodeProjects\ResearchChatBot\.venv\lib\site-packages\pinecone\core\client\rest.py:45: DeprecationWarning: HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).
  return self.urllib3_response.getheader(name, default)


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'no-vectors': {'vector_count': 67}},
 'total_vector_count': 67}

In [50]:
index.fetch(ids=['id1'])

c:\Users\Dom\VSCodeProjects\ResearchChatBot\.venv\lib\site-packages\pinecone\core\client\rest.py:45: DeprecationWarning: HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).
  return self.urllib3_response.getheader(name, default)


{'namespace': '', 'vectors': {}}